In this notebook, we will pass our README dataset through examiner - 1, augmenter, examiner - 2.

# imports

In [ ]:
import numpy as np

# comment the below line if openai is installed in your PC
!pip install openai
# !pip install openai==0.27

import openai

#use the openai key here
API_KEY = ""
openai.api_key = API_KEY

#comment the below lines if using local environemnt
from google.colab import drive
drive.mount('/content/drive')


import time
import pandas as pd
import string
from tenacity import (
    retry,
    stop_after_attempt,
    wait_random_exponential,
)

from datetime import datetime

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.3/70.3 KB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 1.6 MB/s eta 0:00:00
Mounted at /content/drive


# Hyper Parameters

In [ ]:
NUM_EXAMPLES = 100
NUM_AFTER_JOIN = 1000
currentDateTime = datetime.now().strftime("%m-%d-%Y %H-%M-%S %p")

#set locations of all the README datasets
README_v1_PATH = f"/content/drive/MyDrive/final_data.csv"
README_v2_PATH = f"/content/drive/MyDrive/README_v2{currentDateTime}.csv"
README_v3_PATH = f"/content/drive/MyDrive/README_v3{currentDateTime}.csv"
README_v4_PATH = f"/content/drive/MyDrive/README_v4{currentDateTime}.csv"
README_v5_PATH = f"/content/drive/MyDrive/README_v5{currentDateTime}.csv"
README_v6_PATH = f"/content/drive/MyDrive/README_v6{currentDateTime}.csv"


# Prompts


In [ ]:
#this examiner prompt does not use context

examiner = """
Decide whether the general definition is correct.

If we can generate the lay definition from the general definition then answer is yes.

term : mg
general definition : this is short for milligram which is 1/1000 of a gram usually considered a small amount.
lay definition : A tiny amount of something, usually a drug.
answer : yes

term : vitamin c
general definition : ['A nutrient that the body needs in small amounts to function and stay healthy. Vitamin C helps fight infections, heal wounds, and keep tissues healthy. It is an antioxidant that helps prevent cell damage caused by free radicals (highly reactive chemicals). Vitamin C is found in all fruits and vegetables, especially citrus fruits, strawberries, cantaloupe, green peppers, tomatoes, broccoli, leafy greens, and potatoes. It is water-soluble (can dissolve in water) and must be taken in every day. Vitamin C is being studied in the prevention and treatment of some types of cancer.']
lay definition : A nutrient needed by the body to form and maintain bones, blood vessels, and skin.
answer : yes

term : nodule
general definition : ['A small lump, swelling or collection of tissue.']
lay definition : A growth or lump that may be cancerous or not.
answer : yes

term : qd
general definition : ['Occurring or done each day.']
lay definition : Every day.
answer : yes

If the general definition contains many words from the term then answer is no.

term : prochlorperzine
general definition : ['prochlorperzine', ' ']
lay definition : A drug used to prevent or reduce nausea and vomiting.
answer : no

term : mg
general definition : ['mg']
lay definition : A tiny amount of something, usually a drug.
answer : no

If the lay definition can not be generated by the general definition then answer is no.

term : Virt - Vite
general definition : ['Virt', ' - ', 'The determination of the amount of Vitamin E present in a sample.']
lay definition : A mix of vitamins. It provides vitamin B-6, vitamin B-12 and folic acid to people who do not have enough of these for good health.
answer : no

"""

# README data analysis

In [ ]:
df_readme = pd.read_csv(README_v1_PATH)
df_readme.head()

,Unnamed: 0.1,index,Unnamed: 0,id,text_to_annotate,start,end,ann_text,definition,split_print,UMLS+others
0,0,0,0,98791999,Virt - Vite 2.5 - 25 - 1 MG Oral Tablet Vitami...,0.0,3.0,Virt - Vite,"A mix of vitamins. It provides vitamin B-6, vi...","['Virt', ' - ', 'The determination of the amou...","['Virt', ' - ', 'Vite']"
1,1,2,2,98791999,Virt - Vite 2.5 - 25 - 1 MG Oral Tablet Vitami...,8.0,9.0,MG,"A tiny amount of something, usually a drug.",['A measure of weight. A milligram is approxim...,"['MG', ' ']"
2,2,3,3,98791999,Virt - Vite 2.5 - 25 - 1 MG Oral Tablet Vitami...,9.0,10.0,Oral,Taken by mouth.,['A substance intended for administration thro...,['Oral']
3,3,4,4,98791999,Virt - Vite 2.5 - 25 - 1 MG Oral Tablet Vitami...,10.0,11.0,Tablet,A pill.,['A tablet intended for oral administration.'],['Tablet']
4,4,5,5,98791999,Virt - Vite 2.5 - 25 - 1 MG Oral Tablet Vitami...,11.0,14.0,Vitamin B complex,A group of vitamins and nutrients. The body n...,"['A group of water-soluble vitamins, some of w...",['Vitamin B complex']


In [ ]:
print(len(df_readme))

308242


In [ ]:
df_readme.tail()

,Unnamed: 0.1,index,Unnamed: 0,id,text_to_annotate,start,end,ann_text,definition,split_print,UMLS+others
308237,308237,348328,348328,1511235,This laboratory is certified under the Clinica...,14.0,15.0,CLIA,Rules that make laboratory testing better.,['A Federal law establishing quality standards...,['CLIA']
308238,308238,348330,348330,1511235,This laboratory is certified under the Clinica...,24.0,25.0,clinical,Having to do with a patient's illness or treat...,['Having to do with the examination and treatm...,['clinical']
308239,308239,348331,348331,1511234,"Pursuant to the requirements of CLIA , ASR 's ...",5.0,6.0,CLIA,Rules that make laboratory testing better.,['A Federal law establishing quality standards...,"['CLIA', ' ']"
308240,308240,348332,348332,1511234,"Pursuant to the requirements of CLIA , ASR 's ...",7.0,8.0,ASR,Materials used to conduct lab tests.,['Gastrin/cholecystokinin type B receptor (447...,['ASR']
308241,308241,348333,348333,1511234,"Pursuant to the requirements of CLIA , ASR 's ...",12.0,13.0,laboratory,Tests.,['Facilities equipped to carry out investigati...,['laboratory']


In [ ]:
df_v1 = df_readme.drop(columns = ['Unnamed: 0'])
print(df_v1.columns)

Index(['Unnamed: 0.1', 'index', 'id', 'text_to_annotate', 'start', 'end',
       'ann_text', 'definition', 'split_print', 'UMLS+others'],
      dtype='object')


# Examiner 1

text_to_annotate = context

ann_text = term

definition = lay definition

split_print = general definition

UMLS+others = term split into words based on definitions available in UMLS



Here we are using only the term, lay definition and the general definition. That is we are not using any context here for cleaning as context would not add significantly to the response of the gpt-3. This is because the lay definition is sufficiant for gpt3 to check if the general definiton from UMLS is valid.

If you want to also split the term based on the UMLS availability use "UMLS+others" column in place of "ann_text" column

In [ ]:
# df_v1 = df_v1[['text_to_annotate','ann_text',"definition","split_print"]]    #use this if you want to include the context
df_v1 = df_v1[['ann_text',"definition","split_print"]]

In [ ]:
# df_v1 = df_v1.drop_duplicates(subset = ['ann_text',"definition","split_print"])
df_v1 = df_v1.drop_duplicates()
df_v1.head()

,ann_text,definition,split_print
0,Virt - Vite,"A mix of vitamins. It provides vitamin B-6, vi...","['Virt', ' - ', 'The determination of the amou..."
1,MG,"A tiny amount of something, usually a drug.",['A measure of weight. A milligram is approxim...
2,Oral,Taken by mouth.,['A substance intended for administration thro...
3,Tablet,A pill.,['A tablet intended for oral administration.']
4,Vitamin B complex,A group of vitamins and nutrients. The body n...,"['A group of water-soluble vitamins, some of w..."


In [ ]:
df_v1 = df_v1.reset_index(drop = True)

print(len(df_v1))
print("unique datapoints in total are about 51K without context")

51623
unique datapoints in total are about 51K without context


In [ ]:
df_v1 = df_v1.head(100)

In [ ]:
print(len(df_v1))

100


In [ ]:
df_v1.to_csv("first_100.csv")

In [ ]:
def data_gen1(df,i):
  # c = df.iloc[i]['text_to_annotate']
  t = df.iloc[i]['ann_text']
  g_d = df.iloc[i]['split_print']
  l_d = df.iloc[i]['definition']
  # context = "context : " + c
  term = "term : " + t
  general_definition = "general definition : " + g_d
  lay_def = "lay definition : " + l_d
  # custom_prompt = context + "\n" + term + "\n" + general_definition + "\n"+lay_def + "\n"
  custom_prompt = term + "\n" + general_definition + "\n"+lay_def + "\n" + "answer : "
  return custom_prompt


In [ ]:
@retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(6))
def output_examiner1(prompt_t,custom_p,max_t = 1,temp = 0):
  prompt = prompt_t + custom_p
  completion = openai.Completion.create(engine="text-davinci-003", prompt=prompt, max_tokens=max_t, temperature=temp)

  # print("Generated Answer..")
  generated = completion.choices[0].text
  return generated

In [ ]:
"""
df_used is the dataframe used = README_v1 = df_v1
num = number of samples processed = NUM_EXAMPLES
initial prompt = examiner prompt
"""

def reliable(df_used,num,initial_prompt,wait = 0,verbose =False):
  df_used["reliable"] = ""
  for i in range(num):
    if verbose == True and i % 10 == 0:
      print(i)
    custom = data_gen1(df_used,i)
    df_used.at[i,"reliable"] = output_examiner1(initial_prompt,custom)
    time.sleep(wait)

  return df_used


In [ ]:
df_v1 = reliable(df_v1,NUM_EXAMPLES,examiner,0,True)

0
10
20
30
40
50
60
70
80
90


In [ ]:
df_v1.head(10)

,ann_text,definition,split_print,reliable
0,Virt - Vite,"A mix of vitamins. It provides vitamin B-6, vi...","['Virt', ' - ', 'The determination of the amou...",no
1,MG,"A tiny amount of something, usually a drug.",['A measure of weight. A milligram is approxim...,no
2,Oral,Taken by mouth.,['A substance intended for administration thro...,yes
3,Tablet,A pill.,['A tablet intended for oral administration.'],no
4,Vitamin B complex,A group of vitamins and nutrients. The body n...,"['A group of water-soluble vitamins, some of w...",no
5,folic acid,A B vitamin.,['A member of the vitamin B family that stimul...,no
6,vitamin c,A nutrient needed by the body to form and main...,['A nutrient that the body needs in small amou...,yes
7,tab,A pill.,"['A tablet intended for oral administration.',...",no
8,po,By mouth.,['po'],no
9,qd,Every day.,['Occurring or done each day.'],yes


In [ ]:

df_v1.to_csv(f"/content/drive/MyDrive/examiner1_output{currentDateTime}.csv")

In [ ]:
df_v2 = df_v1[df_v1['reliable'] == " yes"]
df_v3 = df_v1[df_v1['reliable'] == " no"]

df_v2 = df_v2.reset_index(drop = True)
df_v3 = df_v3.reset_index(drop = True)

In [ ]:
df_v2.to_csv(README_v2_PATH) # good data or reliable data
df_v3.to_csv(README_v3_PATH) # bad data or unreliable data

# Augmenter

Adding context to the datapoints to improve general definitions created by ChatGPT

In [ ]:
df_v3 = pd.read_csv(README_v3_PATH)

In [ ]:
df_v3.head()

,Unnamed: 0,ann_text,definition,split_print,reliable
0,0,Virt - Vite,"A mix of vitamins. It provides vitamin B-6, vi...","['Virt', ' - ', 'The determination of the amou...",no
1,1,MG,"A tiny amount of something, usually a drug.",['A measure of weight. A milligram is approxim...,no
2,2,Tablet,A pill.,['A tablet intended for oral administration.'],no
3,3,Vitamin B complex,A group of vitamins and nutrients. The body n...,"['A group of water-soluble vitamins, some of w...",no
4,4,folic acid,A B vitamin.,['A member of the vitamin B family that stimul...,no


In [ ]:
df_v3_withcontext = df_v3.merge(df_readme,left_on=["ann_text","definition","split_print"],right_on = ["ann_text","definition","split_print"])

In [ ]:
len(df_v3_withcontext)

4114

In [ ]:
df_v3_withcontext.head(10)

,Unnamed: 0_x,ann_text,definition,split_print,reliable,Unnamed: 0.1,index,Unnamed: 0_y,id,text_to_annotate,start,end,UMLS+others
0,0,Virt - Vite,"A mix of vitamins. It provides vitamin B-6, vi...","['Virt', ' - ', 'The determination of the amou...",no,0,0,0,98791999,Virt - Vite 2.5 - 25 - 1 MG Oral Tablet Vitami...,0.0,3.0,"['Virt', ' - ', 'Vite']"
1,1,MG,"A tiny amount of something, usually a drug.",['A measure of weight. A milligram is approxim...,no,1,2,2,98791999,Virt - Vite 2.5 - 25 - 1 MG Oral Tablet Vitami...,8.0,9.0,"['MG', ' ']"
2,1,MG,"A tiny amount of something, usually a drug.",['A measure of weight. A milligram is approxim...,no,161,205,205,96764483,[ * * PERSON * * ] Linked Records Name DOB ID ...,636.0,637.0,"['MG', ' ']"
3,1,MG,"A tiny amount of something, usually a drug.",['A measure of weight. A milligram is approxim...,no,247,300,300,96592550,PAST MEDICAL / SURGICAL HISTORY PMH Other hist...,209.0,210.0,"['MG', ' ']"
4,1,MG,"A tiny amount of something, usually a drug.",['A measure of weight. A milligram is approxim...,no,251,304,304,96592550,PAST MEDICAL / SURGICAL HISTORY PMH Other hist...,237.0,238.0,"['MG', ' ']"
5,1,MG,"A tiny amount of something, usually a drug.",['A measure of weight. A milligram is approxim...,no,260,314,314,96592550,PAST MEDICAL / SURGICAL HISTORY PMH Other hist...,284.0,285.0,"['MG', ' ']"
6,1,MG,"A tiny amount of something, usually a drug.",['A measure of weight. A milligram is approxim...,no,264,318,318,96592550,PAST MEDICAL / SURGICAL HISTORY PMH Other hist...,312.0,313.0,"['MG', ' ']"
7,1,MG,"A tiny amount of something, usually a drug.",['A measure of weight. A milligram is approxim...,no,603,691,691,95247445,MEDICATIONS CURRENT MEDICATION ORDERS Name Dos...,17.0,18.0,"['MG', ' ']"
8,1,MG,"A tiny amount of something, usually a drug.",['A measure of weight. A milligram is approxim...,no,618,709,709,95247445,MEDICATIONS CURRENT MEDICATION ORDERS Name Dos...,145.0,146.0,"['MG', ' ']"
9,1,MG,"A tiny amount of something, usually a drug.",['A measure of weight. A milligram is approxim...,no,625,718,718,95247445,MEDICATIONS CURRENT MEDICATION ORDERS Name Dos...,201.0,202.0,"['MG', ' ']"


In [ ]:
df_v3_withcontext.to_csv(f"/content/drive/MyDrive/postjoin{currentDateTime}.csv")

Generating the general definitions

In [ ]:
#join the context to the (term,laydef pair). ---> 180K dataset

In [ ]:
system_prompt = "your job is to generate a general definition of the term for a professional medical student"

In [ ]:
@retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(6))
def output_term(prompt_t,system_prompt,max_t = 50,temp = 0.7):
  completion = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": ""},
        {"role": "user", "content": "term : incisional."},
        {"role": "assistant", "content": "general definition : An intentional cut made to an individual's body with the intent of performing a diagnostic or therapeutic intervention."},
        {"role": "user", "content": "term : PO"},
        {"role": "assistant", "content": "general definition : Of, or relating to, or affecting, or for use in the mouth.."},
        {"role": "user", "content": prompt_t}
    ]
)


  content = completion.choices[0].message.content
  role = completion.choices[0].message.role

  return content

In [ ]:
@retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(6))
def data_gen_augmenter(df,i,verbose = False,wait = 0,context = False, term = True, g_d = False, l_d = False):
  if verbose == True and i%100 == 0:
    print(i)
  c = df.iloc[i]['text_to_annotate']
  t = df.iloc[i]['ann_text']
  g_d = df.iloc[i]['split_print']
  l_d = df.iloc[i]['definition']
  context_1 = "context : " + c
  term_1 = "term : " + t
  general_definition = "general definition : " + g_d
  lay_definition = "lay definition : " + l_d

  custom_prompt = ""

  if(term == True):
    custom_prompt += term_1

  if(context == True):
    custom_prompt += context_1 +"\n"

  if(g_d == True):
    custom_prompt += general_definition +"\n"

  if(l_d == True):
    custom_prompt += lay_definition +"\n"

  df.at[i,"gen_def_chatGPT"] = output_term(custom_prompt,system_prompt)


In [ ]:
df_v4 = df_v3_withcontext.copy()

In [ ]:
df_v4['gen_def_chatGPT'] = ""

for i in range(NUM_AFTER_JOIN):
  # print(i)
  data_gen_augmenter(df_v4,i,True,0,True,True,False,False)
  # print(temp)

0
100
200
300
400
500
600
700
800
900
1000


KeyboardInterrupt: ignored

In [ ]:
df_v4.to_csv(README_v4_PATH)

# Examiner 2

In [ ]:
def data_gen2(df,i):
  # c = df.iloc[i]['text_to_annotate']
  t = df.iloc[i]['ann_text']
  g_d = df.iloc[i]['gen_def_chatGPT']
  l_d = df.iloc[i]['definition']
  # context = "context : " + c
  term = "term : " + t
  general_definition = "general definition : " + g_d
  lay_def = "lay definition : " + l_d
  # you can add answer to spend fewer tokens
  # custom_prompt = context + "\n" + term + "\n" + general_definition + "\n"+lay_def + "\n"
  custom_prompt = term + "\n" + general_definition + "\n"+lay_def + "\n" + "answer : "
  return custom_prompt

In [ ]:

@retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(6))
def reliable(df_used,num,initial_prompt,wait = 0,verbose =False):
  df_used["reliable"] = ""
  for i in range(num):
    if verbose == True and i % 100 == 0:
      print(i)
    custom = data_gen2(df_used,i)
    df_used.at[i,"reliable"] = output_examiner1(initial_prompt,custom)
    time.sleep(wait)

  return df_used


In [ ]:
df_v4 = reliable(df_v4,NUM_AFTER_JOIN,examiner,0,True)

0
100
200
300
400
500
600
700
800
900


In [ ]:
df_v4.to_csv(f"/content/drive/MyDrive/examiner2_output{currentDateTime}.csv")

In [ ]:
df_v5 = df_v4[df_v4['reliable'] == " yes"]
df_v6 = df_v4[df_v4['reliable'] == " no"]

df_v5 = df_v5.reset_index(drop = True)
df_v6 = df_v6.reset_index(drop = True)

In [ ]:
df_v5.to_csv(README_v5_PATH)
df_v6.to_csv(README_v6_PATH)